In [264]:
from sympy import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Enable the mathjax printer
init_printing(use_latex='mathjax')

In [265]:
class Robot:
    def __init__(self, configuration:str, dh_table:Matrix, name:str='Robot', **matrices):
        """
        Initialize the robot

        Parameters:
        configuration: str
            Set the joint types: 'RRR', 'RRP', ...
        
        dh_table: sympy.Matrix
            Table with Denavit-Hartenberg parameters.
            Use (theta, d, a, alpha) for each joint.
        
        name: str
            Define a name for the robot.
        
        matrices: sympy.Matrix
            masses:     Matrix with the mass of each link: [m1, m2, m3].T (nx1)
            r_cis:      Matrix with the com of each link (local): [ci1, ci2, ci3].T (nx1)
            inertias:   Matrix with the inertia of each link: [[I11:4], [I21:4], [I31:4]] (3x3)
            g_vec:      Matrix with the gravity vector: [0 0 -g].T (3x1)        
        """
        self.name = name
        self.joint_types = list(configuration)
        self.dof = len(self.joint_types)

        self.joints = pd.DataFrame(columns=['type', 'theta', 'd', 'a', 'alpha'])
        self.t = symbols('t')
        self.q = [Function(f'q{i+1}')(self.t) for i in range(self.dof)]
        self.dq = [qi.diff(self.t) for qi in self.q]
        self.ddq = [dqi.diff(self.t) for dqi in self.dq]
        self.set_matrices(**matrices)
        self.set_dh_params(dh_table)
        self.get_joints()
        self.compute()

    def set_matrices(self, **matrices):
        """
        Set the mass, center of mass, inertia and gravity matrices

        Parameters:
        matrices: sympy.Matrix
            masses:     Matrix with the mass of each link: [m1, m2, m3].T (nx1)
            r_cis:      Matrix with the com of each link (local): [ci1, ci2, ci3].T (nx1)
            inertias:   Matrix with the inertia of each link: [[I11:4], [I21:4], [I31:4]] (3x3)
            g_vec:      Matrix with the gravity vector: [0 0 -g].T (3x1) 

        Default:
            All parameters are defined symbolic.
        """
        self.masses_symbols = Matrix(symbols(f'm1:{self.dof+1}'))
        self.masses = matrices.get('masses', self.masses_symbols)
        self.r_cis_local = matrices.get('r_cis', [Matrix(symbols(f'r_{i+1}x:z')) for i in range(self.dof)])
        self.inertias_symbols = [symbols(f'I{i+1}_1:4_1:4') for i in range(3)]
        self.inertias = matrices.get('inertias', [Matrix(3,3,self.inertias_symbols[i]) for i in range(self.dof)])
        self.g_symbol = symbols('g')
        self.g_vec = matrices.get('g_vec', Matrix([0, 0, -self.g_symbol]))
    
    def set_dh_params(self, dh_table):
        """
        Set the Denavit Hartenberg parameters

        Parameters:
        dh_table: list of lists
            Each list represent the DH-parameters of a joint in the order (theta, d, a, alpha)
        """
        self.thetas = dh_table[:,0]
        self.ds = dh_table[:,1]
        self._as_ = dh_table[:,2]
        self.alphas = dh_table[:,3]
    
    def get_joints(self):
        """
        Generates the DH Table amd substitutes the symbolic q with symbolic function q(t)
        """
        q = symbols(f'q1:{self.dof+1}')
        for tp in self.joint_types:
            if tp == 'R': self.thetas = self.thetas.subs(zip(q,self.q))
            if tp == 'P': self.ds = self.ds.subs(zip(q,self.q))
        self.joints = pd.DataFrame(list(zip(self.joint_types, Matrix(self.thetas), self.ds, self._as_, self.alphas)), 
                                   columns=['type', 'theta', 'd', 'a', 'alpha'], 
                                   index=np.arange(self.dof)+1)
    
    def compute(self):
        """
        Computes all the kinematics and dynamics and saves the information in attributes
        """
        # Kynematics
        self.dh_matrices = [self.compute_dh_matrix(joint_index=i+1) for i in range(self.dof)]
        self.base_to_joint = [self.calculate_base_to_joint(joint_end=i+1) for i in range(self.dof)]
        self.base_to_end_effector = self.base_to_joint[-1]
        self.jacobian = self.calculate_jacobian()
        # Center of Mass
        self.r_cis_global = [self.calculate_r_i_ci(joint_index=i+1) for i in range(self.dof)]
        self.jacobian_ci = self.calculate_jacobian(com=True)
        # Dynamics
        self.inertia_matrix = self.calculate_inertia_matrix()
        self.coriolis_matrix = self.calculate_coriolis_matrix()
        self.gravity_vector = self.calculate_gravity_vector()
        self.torques = self.compute_inverse_dynamics()

    def compute_dh_matrix(self, joint_index:int):
        """
        Compute the Denavit-Hartenberg transformation matrix for a given joint.

        Parameters:
        joint_index: int 

        Returns:
        sympy.Matrix
            Denavit-Hartenberg transformation matrix
        """
        assert joint_index > 0, "Must be positive."
        i = joint_index - 1
        theta, d, a, alpha = self.thetas[i], self.ds[i], self._as_[i], self.alphas[i]
        T = Matrix([[cos(theta), -sin(theta)*cos(alpha), sin(theta)*sin(alpha), a*cos(theta)],
                     [sin(theta), cos(theta)*cos(alpha), -cos(theta)*sin(alpha), a*sin(theta)],
                     [0, sin(alpha), cos(alpha), d],
                     [0, 0, 0, 1]])
        return T
    
    def calculate_joint_to_joint(self, joint_start:int, joint_end:int):
        """
        Computes the transformation matrix from a starting joint to an ending joint.

        Parameters:
        joint_start: int
            Index of starting joint.

        end_joint: int
            Index of ending joint.

        Returns:
        sympy.Matrix
            Transformation matrix from starting to ending joint
        """
        assert joint_start > 0, "Must be positive."
        assert joint_end >= joint_start, "End must be greater than start."
        result = eye(self.dof + 1)
        for i in range(joint_start, joint_end+1):
            result = result @ self.dh_matrices[i-1]
        return simplify(result)
    
    def calculate_base_to_joint(self, joint_end):
        """
        Computes the transformation matrix from base to an ending joint
        """
        return self.calculate_joint_to_joint(joint_start=1, joint_end=joint_end)
    
    def calculate_base_to_end_effector(self):
        """
        Computes the transformation matrix from base to end-effector.
        """
        return self.calculate_base_to_joint(joint_end=self.dof)
    
    def get_rotation(self, matrix):
        """
        Selects the Rotation Matrix from the Transformation Matrix.
        """
        return matrix[:self.dof,:self.dof]
    
    def get_translation(self, matrix):
        """
        Selects the Translation Matrix from the Transformation Matrix.
        """
        return matrix[:self.dof,-1:]

    def z_i_minus_one(self, joint_index):
        """
        Calculates the orientation of the z axis for the desired joint.
        This will be used to calculate the Jacobian.

        Parameters:
        joint_index: int
        """
        result = eye(self.dof)
        for i in range(1, joint_index):
            result = result @ self.get_rotation(self.dh_matrices[i-1])
        # result = self.get_rotation(self.base_to_joint[joint_index-1])
        result = result @ Matrix([0, 0, 1])
        return simplify(result)
    
    def r_i_minus_one_to_n(self, joint_index, joint_ci=None):
        """
        Calculates the position of the desired joint relative to the end-effector.
        This will be used to calculate the Jacobian.

        Parameters:
        joint_index: int 

        joint_ci: int
            Considers the center of mass of joint {joint_ci}
        """
        com = joint_ci is not None
        if not com:
            return self.get_translation(self.base_to_end_effector - self.base_to_joint[joint_index-1]) \
                    if joint_index > 1 else self.get_translation(self.base_to_end_effector)
        else:
            return self.r_cis_global[joint_ci-1] - self.get_translation(self.base_to_joint[joint_index - 1])

    def calculate_r_i_ci(self, joint_index):
        """
        The same as self.r_i_minus_one_to_n(), but considering the position of the center of mass of the links.
        """
        r_local = Matrix.vstack(self.r_cis_local[joint_index-1], Matrix([1]))
        r_ci_global = self.base_to_joint[joint_index-1] @ r_local
        return simplify(r_ci_global[:-1,:])

    def calculate_jacobian_col_i(self, joint_index:int, joint_ci=None):
        """
        Computes the columns of the jacobian matrix.

        Parameters:
        joint_index: int
            Defines which column to be computed.

        joint_ci: int (Optional)
            Defines which link (center of mass) to consider.
        
        Returns:
        sympy.Matrix
            Single column of the jacobian matrix (6 x 1)
        """
        com = joint_ci is not None
        if com:
            if joint_ci < joint_index:
                return Matrix(6*[0])
        joint_type = self.joint_types[joint_index-1]
        if joint_type == 'R':
            J_vi = Matrix(np.cross(self.z_i_minus_one(joint_index).T, self.r_i_minus_one_to_n(joint_index, joint_ci).T)).T
            J_wi = self.z_i_minus_one(joint_index)
        elif joint_type == 'P':
            J_vi = self.z_i_minus_one(joint_index)
            J_wi = Matrix([0, 0, 0])
        J_i = Matrix(np.vstack((J_vi, J_wi)))
        return simplify(J_i)

    def calculate_jacobian(self, com=False):
        """
        Calculates the jacobian matrix of the robot. 

        Parameters:
        com: bool
            If False, relative to end-effector. (Default)
            If True, relative to center of mass of each link.

        Returns:
        sympy.Matrix
            Jacobian matrix (6 x n).
        """
        if not com:
            return Matrix(np.hstack(tuple((self.calculate_jacobian_col_i(i+1) for i in range(self.dof)))))
        else:
            return [Matrix(np.hstack(tuple((self.calculate_jacobian_col_i(j+1, i+1) 
                    for j in range(self.dof)))))
                    for i in range(self.dof)]

    def calculate_inertia_matrix(self):
        """
        Calculates the inertia matrix of the robot.

        Returns:
        sympy.Matrix
            Inertia matrix (3 x 3).
        """
        inertia_matrix = zeros(self.dof, self.dof)

        for i in range(self.dof):
            jv = self.jacobian_ci[i][:3, :]
            jw = self.jacobian_ci[i][3:, :]
            inertia_matrix += self.masses[i] * (jv.T @ jv) + (jw.T @ self.inertias[i] @ jw)

        return simplify(inertia_matrix)

    def calculate_coriolis_matrix(self):
        """
        Calculate the Coriolis matrix C(q).

        Returns:
        sympy.Matrix
            Coriolis matrix (n x n).
        """
        n = self.dof
        # q = symbols(f'q1:{n+1}')
        # dq = symbols(f'dq1:{n+1}')

        M = self.inertia_matrix
        C = zeros(n, n)
        for i in range(n):
            for j in range(n):
                for k in range(n):
                    c_ijk = 0.5 * (diff(M[i, k], self.q[j]) + diff(M[i, j], self.q[k]) - diff(M[j, k], self.q[i]))
                    C[i, j] += c_ijk * self.dq[k]

        return simplify(C)

    def calculate_gravity_vector(self):
        """
        Calculate the gravity vector G(q) by differentiating the total potential energy.

        Returns:
        sympy.Matrix
            Gravity vector.
        """
        U = 0
        for i, com in enumerate(self.r_cis_global):
            U += self.masses[i] * self.g_vec.dot(com)
        # Differentiating the total potential energy
        G = Matrix([diff(U, q_i) for q_i in self.q])
        return G
    
    def compute_inverse_dynamics(self):
        """
        Computes the torque and forces in each joint.

        Returns:
        sympy.Matrix
            Torques/forces (n x 1)
        """
        self.q_d = [Function(f'q_{i+1}d')(self.t) for i in range(self.dof)]
        self.dq_d = [qi.diff(self.t) for qi in self.q_d]
        self.ddq_d = [dqi.diff(self.t) for dqi in self.dq_d]

        M = self.inertia_matrix
        C = self.coriolis_matrix
        G = self.gravity_vector
        tau = M.subs(zip(self.q,self.q_d)) @ Matrix(self.ddq_d) + \
              C.subs(zip(self.q,self.q_d)).subs(zip(self.dq, self.dq_d)) @ Matrix(self.dq_d) + \
              G.subs(zip(self.q,self.q_d))
        return simplify(tau)

    def eval_dinamics(self, tau, q_d=None, dq_d=None, ddq_d=None):
        """
        Substitutes the symbolic variables with numeric or other symbolic values.

        Parameters:
        q_d:
            Desired position/trajectory.

        dq_d:
            Desired velocity.
        
        ddq_d:
            Desired acceleration.
        
        Note:
            Be aware of the order of operations implemented and check the result.
        """
        # Importante manter esta ordem!
        if ddq_d: tau = tau.subs(zip(self.ddq_d, ddq_d))
        if dq_d: tau = tau.subs(zip(self.dq_d, dq_d))
        if q_d: tau = tau.subs(zip(self.q_d, q_d))
        return tau.simplify()
    
    def eval_matrix(self, matrix, q=None, dq=None, ddq=None):
        # Importante manter esta ordem!
        if ddq: matrix = matrix.subs(zip(self.ddq, ddq))
        if dq: matrix = matrix.subs(zip(self.dq, dq))
        if q: matrix = matrix.subs(zip(self.q, q))
        return matrix.simplify()

In [266]:
# == Define Configuration == #
configuration = 'RPR'
dof = len(configuration)


# == Define Masses == #
m1, m2, m3 = Matrix(symbols(f'm_1:4'))
masses = Matrix([m1, m2, m3])
L1, L2, L3 = symbols('L_1:4')


# == Define inertias == #
inertias = [diag(*symbols('I_1x:z')), diag(0,0,0), diag(*symbols('I_3x:z'))]


# == Define Center of Mass (local) == #
# cm1, cm2, cm3 = symbols(f'r_1:4')
# r_cis = [Matrix([0, 0, cm]) for cm in [cm1, cm2, cm3]]
r_cis = [Matrix([0, 0, cm]) for cm in [L1/2, L2/2, L3/2]] 


# == Define gravity vector (base) == #
g_vec = Matrix([0, 0, -symbols('g')])
# g_vec = Matrix([0, 0, -9.81])


# == Define DH parameters == #
# q1, q2, q3 = symbols('q_1:4')
tsym = symbols('t')
q1, q2, q3 = [Function(f'q{i+1}')(tsym) for i in range(3)]

# == DH Table == #
dh_table = Matrix(
    [[q1, 0, 0, -pi/2],
    [0, q2, 0, pi/2],
    [q3, 0, L3, 0]]
)

# == Creating the Robot == #
r = Robot(configuration='RPR', dh_table=dh_table, masses=masses, r_cis=r_cis, inertias=inertias)    # masses=masses, r_cis=r_cis, g_vec=g_vec, inertias=inertias
r.joints

,type,theta,d,a,alpha
1,R,q1(t),0,0,-pi/2
2,P,0,q2(t),0,pi/2
3,R,q3(t),0,L_3,0


In [ ]:
# DH Parameters
# r.thetas, r.ds, r._as_, r.alphas

⎛                        ⎡-π ⎤⎞
⎜                        ⎢───⎥⎟
⎜⎡q₁(t)⎤  ⎡  0  ⎤  ⎡0 ⎤  ⎢ 2 ⎥⎟
⎜⎢     ⎥  ⎢     ⎥  ⎢  ⎥  ⎢   ⎥⎟
⎜⎢  0  ⎥, ⎢q₂(t)⎥, ⎢0 ⎥, ⎢ π ⎥⎟
⎜⎢     ⎥  ⎢     ⎥  ⎢  ⎥  ⎢ ─ ⎥⎟
⎜⎣q₃(t)⎦  ⎣  0  ⎦  ⎣L₃⎦  ⎢ 2 ⎥⎟
⎜                        ⎢   ⎥⎟
⎝                        ⎣ 0 ⎦⎠

In [ ]:
# DOF, q, dq, ddq, 
# r.dof, r.q, r.dq, r.ddq

⎛                                                             ⎡ 2           2  ↪
⎜                          ⎡d          d          d        ⎤  ⎢d           d   ↪
⎜3, [q₁(t), q₂(t), q₃(t)], ⎢──(q₁(t)), ──(q₂(t)), ──(q₃(t))⎥, ⎢───(q₁(t)), ─── ↪
⎜                          ⎣dt         dt         dt       ⎦  ⎢  2           2 ↪
⎝                                                             ⎣dt          dt  ↪

↪           2        ⎤⎞
↪          d         ⎥⎟
↪ (q₂(t)), ───(q₃(t))⎥⎟
↪            2       ⎥⎟
↪          dt        ⎦⎠

In [269]:
# Masses, Inertias, gravity vector
# r.masses, r.inertias, r.g_vec

In [270]:
# Position of center of mass (local and global)
# r.r_cis_local, r.r_cis_global

In [271]:
# Transition Matrix: i-1 -> i
# r.dh_matrices

In [ ]:
# Transition Matrix: 0 -> i
# r.base_to_joint

⎡⎡cos(q₁(t))  0   -sin(q₁(t))  0⎤  ⎡cos(q₁(t))  -sin(q₁(t))  0  -q₂(t)⋅sin(q₁( ↪
⎢⎢                              ⎥  ⎢                                           ↪
⎢⎢sin(q₁(t))  0   cos(q₁(t))   0⎥  ⎢sin(q₁(t))  cos(q₁(t))   0  q₂(t)⋅cos(q₁(t ↪
⎢⎢                              ⎥, ⎢                                           ↪
⎢⎢    0       -1       0       0⎥  ⎢    0            0       1          0      ↪
⎢⎢                              ⎥  ⎢                                           ↪
⎣⎣    0       0        0       1⎦  ⎣    0            0       0          1      ↪

↪ t))⎤  ⎡cos(q₁(t) + q₃(t))  -sin(q₁(t) + q₃(t))  0  L₃⋅cos(q₁(t) + q₃(t)) - q ↪
↪    ⎥  ⎢                                                                      ↪
↪ )) ⎥  ⎢sin(q₁(t) + q₃(t))  cos(q₁(t) + q₃(t))   0  L₃⋅sin(q₁(t) + q₃(t)) + q ↪
↪    ⎥, ⎢                                                                      ↪
↪    ⎥  ⎢        0                    0           1                     0      ↪
↪    ⎥  ⎢                  

In [ ]:
# Transition Matrix: 0 -> n (dof) [from base to end effector]
# r.base_to_end_effector

⎡cos(q₁(t) + q₃(t))  -sin(q₁(t) + q₃(t))  0  L₃⋅cos(q₁(t) + q₃(t)) - q₂(t)⋅sin ↪
⎢                                                                              ↪
⎢sin(q₁(t) + q₃(t))  cos(q₁(t) + q₃(t))   0  L₃⋅sin(q₁(t) + q₃(t)) + q₂(t)⋅cos ↪
⎢                                                                              ↪
⎢        0                    0           1                     0              ↪
⎢                                                                              ↪
⎣        0                    0           0                     1              ↪

↪ (q₁(t))⎤
↪        ⎥
↪ (q₁(t))⎥
↪        ⎥
↪        ⎥
↪        ⎥
↪        ⎦

In [ ]:
# Jacobian (end effector)
# r.jacobian

⎡-L₃⋅sin(q₁(t) + q₃(t)) - q₂(t)⋅cos(q₁(t))  -sin(q₁(t))  0⎤
⎢                                                         ⎥
⎢L₃⋅cos(q₁(t) + q₃(t)) - q₂(t)⋅sin(q₁(t))   cos(q₁(t))   0⎥
⎢                                                         ⎥
⎢                    0                           0       0⎥
⎢                                                         ⎥
⎢                    0                           0       0⎥
⎢                                                         ⎥
⎢                    0                           0       0⎥
⎢                                                         ⎥
⎣                    1                           0       1⎦

In [ ]:
# Jacobian for each center of mass
# r.jacobian_ci

⎡⎡-L₁⋅cos(q₁(t))       ⎤                                                       ↪
⎢⎢───────────────  0  0⎥                                                       ↪
⎢⎢       2             ⎥  ⎡-q₂(t)⋅cos(q₁(t))  -sin(q₁(t))  0⎤  ⎡-L₃⋅sin(q₁(t)  ↪
⎢⎢                     ⎥  ⎢                                 ⎥  ⎢               ↪
⎢⎢-L₁⋅sin(q₁(t))       ⎥  ⎢-q₂(t)⋅sin(q₁(t))  cos(q₁(t))   0⎥  ⎢L₃⋅cos(q₁(t) + ↪
⎢⎢───────────────  0  0⎥  ⎢                                 ⎥  ⎢               ↪
⎢⎢       2             ⎥  ⎢        0               0       0⎥  ⎢               ↪
⎢⎢                     ⎥, ⎢                                 ⎥, ⎢               ↪
⎢⎢       0         0  0⎥  ⎢        0               0       0⎥  ⎢               ↪
⎢⎢                     ⎥  ⎢                                 ⎥  ⎢               ↪
⎢⎢       0         0  0⎥  ⎢        0               0       0⎥  ⎢               ↪
⎢⎢                     ⎥  ⎢                                 ⎥  ⎢               ↪
⎢⎢       0         0  0⎥  ⎣ 

In [276]:
# Inertia Matrix
r.inertia_matrix

⎡                2                                                             ↪
⎢              L₁ ⋅m₁        2         ⎛  2                             2   ⎞  ↪
⎢I_1z + I_3z + ────── + m₂⋅q₂ (t) + m₃⋅⎝L₃  + 2⋅L₃⋅q₂(t)⋅sin(q₃(t)) + q₂ (t)⎠  ↪
⎢                4                                                             ↪
⎢                                                                              ↪
⎢                              L₃⋅m₃⋅cos(q₃(t))                                ↪
⎢                                                                              ↪
⎣                                    I_3z                                      ↪

↪                        ⎤
↪                        ⎥
↪  L₃⋅m₃⋅cos(q₃(t))  I_3z⎥
↪                        ⎥
↪                        ⎥
↪      m₂ + m₃        0  ⎥
↪                        ⎥
↪         0          I_3z⎦

In [277]:
# Coriolis/Centripetal
r.coriolis_matrix

⎡                           d                                                  ↪
⎢1.0⋅L₃⋅m₃⋅q₂(t)⋅cos(q₃(t))⋅──(q₃(t)) + 1.0⋅(m₂⋅q₂(t) + m₃⋅(L₃⋅sin(q₃(t)) + q₂ ↪
⎢                           dt                                                 ↪
⎢                                                                              ↪
⎢                         d                                                    ↪
⎢  - 0.5⋅L₃⋅m₃⋅sin(q₃(t))⋅──(q₃(t)) - 1.0⋅(m₂⋅q₂(t) + m₃⋅(L₃⋅sin(q₃(t)) + q₂(t ↪
⎢                         dt                                                   ↪
⎢                                                                              ↪
⎢                  ⎛                       d                          d        ↪
⎢            L₃⋅m₃⋅⎜- 1.0⋅q₂(t)⋅cos(q₃(t))⋅──(q₁(t)) + 0.5⋅sin(q₃(t))⋅──(q₂(t) ↪
⎣                  ⎝                       dt                         dt       ↪

↪       d                                 d                                    ↪
↪ (t)))⋅──(q₂(t))  - 0.5⋅L₃

In [278]:
# Gravity vector
simplify(r.gravity_vector)

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [279]:
# Torques and Forces (Tau)
r.torques

⎡          2                       2                       2                   ↪
⎢         d                       d                       d                    ↪
⎢1.0⋅I_1z⋅───(q_1d(t)) + 1.0⋅I_3z⋅───(q_1d(t)) + 1.0⋅I_3z⋅───(q_3d(t)) + 0.25⋅ ↪
⎢           2                       2                       2                  ↪
⎢         dt                      dt                      dt                   ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                           

In [280]:
# Symbols
print('Da pra substituir essas variaveis:')
print(r.torques.free_symbols)
print(f'\nSimbolos:')
r.torques.free_symbols

Da pra substituir essas variaveis:
{L_1, m_3, m_1, L_3, t, m_2, I_1z, I_3z}

Simbolos:


{I_1z, I_3z, L₁, L₃, m₁, m₂, m₃, t}

In [281]:
# Substitution and evaluation:

# Define what to substitute
substitution_dict = {
    'm_1':10, 'm2':0, 'm_3':20,
    'I_1x':1e-5, 'I_1y':1e-5, 'I_1z':1e-6,
    'I_3x':1e-4, 'I_3y':1e-4, 'I_3z':1-4,
    'r_1':0.5, 'r_3':0.7,
    'L_1':.5, 'L_2':.6, 'L_3':0.9,
    'g':9.81
}

In [282]:
def coeff_traj(q0, qf, tf):
    D = qf - q0
    a0 = q0
    a1 = 0
    a2 = 0
    a3 = 10 * D / tf**3
    a4 = -15 * D / tf**4
    a5 = 6 * D / tf**5
    return a0, a1, a2, a3, a4, a5

def calc_traj(a, t):
    q   = a[0] + a[1]*t + a[2]*t**2 + a[3]*t**3 + a[4]*t**4 + a[5]*t**5
    qd  = a[1] + 2*a[2]*t + 3*a[3]*t**2 + 4*a[4]*t**3 + 5*a[5]*t**4
    qdd = 2*a[2] + 6*a[3]*t + 12*a[4]*t**2 + 20*a[5]*t**3
    return q, qd, qdd

def des_traj(a, t):
    q = a[0] + a[1]*t + a[2]*t**2 + a[3]*t**3 + a[4]*t**4 + a[5]*t**5
    return q

q0 = np.array([0, np.pi/2, 0])
qf = np.array([np.pi/2, (3/2)*np.pi, np.pi])
tf = 5.0 
tgrid = np.linspace(0, tf, 300)

coeffs = [coeff_traj(q0[i], qf[i], tf) for i in range(r.dof)]
trajs = [lambda t, c=coeff: calc_traj(c, t) for coeff in coeffs]

q_d=[trajs[i](r.t)[0] for i in range(r.dof)]
dq_d=[trajs[i](r.t)[1] for i in range(r.dof)]
ddq_d=[trajs[i](r.t)[2] for i in range(r.dof)]

In [283]:
# Como avaliar os valores de uma matriz (transformar em uma funcao)
matrix = r.coriolis_matrix  # Escolha a matriz: r.inertia_matrix, r.coriolis_matrix
subs_matrix = matrix.subs(substitution_dict).simplify()
t_matrix = r.eval_matrix(matrix=subs_matrix, 
                        q=q_d,
                        dq=dq_d,
                        ddq=ddq_d
                       )
matrix_func = lambdify('t', t_matrix)

In [284]:
# Original
matrix

⎡                           d                                                  ↪
⎢1.0⋅L₃⋅m₃⋅q₂(t)⋅cos(q₃(t))⋅──(q₃(t)) + 1.0⋅(m₂⋅q₂(t) + m₃⋅(L₃⋅sin(q₃(t)) + q₂ ↪
⎢                           dt                                                 ↪
⎢                                                                              ↪
⎢                         d                                                    ↪
⎢  - 0.5⋅L₃⋅m₃⋅sin(q₃(t))⋅──(q₃(t)) - 1.0⋅(m₂⋅q₂(t) + m₃⋅(L₃⋅sin(q₃(t)) + q₂(t ↪
⎢                         dt                                                   ↪
⎢                                                                              ↪
⎢                  ⎛                       d                          d        ↪
⎢            L₃⋅m₃⋅⎜- 1.0⋅q₂(t)⋅cos(q₃(t))⋅──(q₁(t)) + 0.5⋅sin(q₃(t))⋅──(q₂(t) ↪
⎣                  ⎝                       dt                         dt       ↪

↪       d                                 d                                    ↪
↪ (t)))⋅──(q₂(t))  - 0.5⋅L₃

In [285]:
# Com parametros substituidos
subs_matrix

⎡                                            d                                 ↪
⎢1.0⋅(m₂⋅q₂(t) + 20⋅q₂(t) + 18.0⋅sin(q₃(t)))⋅──(q₂(t)) + 18.0⋅q₂(t)⋅cos(q₃(t)) ↪
⎢                                            dt                                ↪
⎢                                                                              ↪
⎢                                                d                          d  ↪
⎢  - 1.0⋅(m₂⋅q₂(t) + 20⋅q₂(t) + 18.0⋅sin(q₃(t)))⋅──(q₁(t)) - 9.0⋅sin(q₃(t))⋅── ↪
⎢                                                dt                         dt ↪
⎢                                                                              ↪
⎢                                     d                          d             ↪
⎢             - 18.0⋅q₂(t)⋅cos(q₃(t))⋅──(q₁(t)) + 9.0⋅sin(q₃(t))⋅──(q₂(t))     ↪
⎣                                     dt                         dt            ↪

↪  d                                                      d                    ↪
↪ ⋅──(q₃(t))  1.0⋅(m₂⋅q₂(t)

In [286]:
# Com variaveis substituidas q(t), dq(t), ddq(t)
t_matrix

⎡ 2 ⎛                   2                                        ⎞ ⎛       ⎛   ↪
⎢t ⋅⎝0.030159289474462⋅t  - 0.30159289474462⋅t + 0.75398223686155⎠⋅⎝1.0⋅m₂⋅⎝0. ↪
⎢                                                                              ↪
⎢          2 ⎛⎛                     2                                        ⎞ ↪
⎢         t ⋅⎝⎝- 0.271433605270158⋅t  + 2.71433605270158⋅t - 6.78584013175395⎠ ↪
⎢                                                                              ↪
⎢                                                         2 ⎛       ⎛          ↪
⎣                                                        t ⋅⎝- 18.0⋅⎝0.0150796 ↪

↪                   5                      4                      3            ↪
↪ 0060318578948924⋅t  - 0.075398223686155⋅t  + 0.251327412287183⋅t  + 1.570796 ↪
↪                                                                              ↪
↪     ⎛ 3 ⎛                    2                                          ⎞⎞   ↪
↪ ⋅sin⎝t ⋅⎝0.00603185789489

In [287]:
# Como funcao: matrix(t)
i = 150
Matrix(matrix_func(t=tgrid[i]))

⎡3.71262313170224⋅m₂ + 94.7984565777427  1.85631156585109⋅m₂ + 37.126231317022 ↪
⎢                                                                              ↪
⎢-1.85631156585112⋅m₂ - 58.330478599306                    0                   ↪
⎢                                                                              ↪
⎣           10.9312503104346                        5.30106182057091           ↪

↪ 6  -10.9312503104346⎤
↪                     ⎥
↪    -5.30106182057091⎥
↪                     ⎥
↪            0        ⎦

In [288]:
# # Outra forma de avaliar a matriz
# ti_matrix = r.eval_matrix(matrix=subs_matrix, 
#                           q=lambdify('t',q)(tgrid[i]),
#                           dq=lambdify('t',dq)(tgrid[i]),
#                           ddq=lambdify('t',ddq)(tgrid[i])
#                        )
# ti_matrix

Análise do máximo:

In [289]:
# Inertia Matrix
M = r.inertia_matrix
subs_M = simplify(M.subs(substitution_dict))
t_M = r.eval_matrix(matrix=subs_M, q=q_d)
M_func = lambdify('t', t_M)
M = [M_func(t=t) for t in tgrid]

# Coriolis Matrix
C = r.coriolis_matrix
subs_C = simplify(C.subs(substitution_dict))
t_C = r.eval_matrix(matrix=subs_C, q=q_d, dq=dq_d)
C_func = lambdify('t', t_C)
C = [C_func(t=t) for t in tgrid]

# Gravity Vector
G = r.gravity_vector
subs_G = simplify(G.subs(substitution_dict))
t_G = r.eval_matrix(matrix=subs_G, q=q_d)
G_func = lambdify('t', t_G)
G = [G_func(t=t) for t in tgrid]

In [290]:
# Maximo Inercia
M[np.argmax([np.linalg.norm(Mi) for Mi in M])]

TypeError: loop of ufunc does not support argument 0 of type Add which has no callable sqrt method

In [ ]:
# Maximo Coriolis
C[np.argmax([np.linalg.norm(Ci) for Ci in C])]

In [ ]:
i = 150
r.eval_matrix(matrix=subs_matrix, 
                q=lambdify('t',q_d)(tgrid[i]),
                dq=lambdify('t',dq_d)(tgrid[i]),
                ddq=lambdify('t',ddq_d)(tgrid[i])
            )

### Controle PID

In [ ]:
# Trajetória desejada
q_ds = np.asarray(lambdify('t',q_d)(tgrid)).T       # Tem que ser (N, 3)
dq_ds = np.asarray(lambdify('t',dq_d)(tgrid)).T
ddq_ds = np.asarray(lambdify('t',ddq_d)(tgrid)).T

# Trajetória real
q = np.zeros_like(q_ds)
dq = np.zeros_like(dq_ds)
e_int = np.zeros(3)

# Ganhos PID
Kp = np.diag([1345.7, 1567.2, 4915.6])
Kd = np.diag([69.55, 80.99, 254.05])
Ki = np.diag([17767, 20691, 64901])
# Ki = np.diag([0, 0, 0])


N = len(tgrid)
dt = tgrid[1] - tgrid[0]
for k in range(N-1):
    e = q_ds[k] - q[k]
    edot = dq_ds[k] - dq[k]
    e_int += e * dt

    M = np.array(r.eval_matrix(matrix=subs_M, q=list(q[k])), dtype=float)
    C = np.array(r.eval_matrix(matrix=subs_C, q=list(q[k]), dq=list(dq[k])), dtype=float)
    G = np.array(r.eval_matrix(matrix=subs_G, q=list(q[k])), dtype=float).flatten()
    print('M', M)
    print('C', C)
    print('G', G)
    # Cálculo de torque PID
    tau = (M @ ddq_ds[k] + C @ dq_ds[k] + G - Kp @ e - Kd @ edot - Ki @ e_int)
    # tau = - Kp @ e - Kd @ edot - Ki @ e_int

    # Simula próxima aceleração
    # rhs = tau - C @ dq[k] - G
    # qdd = np.linalg.solve(M, rhs)
    qdd = np.linalg.inv(M) @ (tau - C @ dq[k] - G)
    dq[k+1] = dq[k] + qdd * dt
    q[k+1] = q[k] + dq[k+1] * dt

TypeError: Cannot convert expression to float

### Extra (old)

In [ ]:
# # Choose a function or expression
# f = r.torques

# # Options (basicamente equivalentes)
# tau = f.subs(substitution_dict).simplify()        # For substitution (can be other symbolic expression, i.e.)
# # f.evalf(subs=substitution_dict).simplify()  # For numeric substitution and evaluation
# tau

In [ ]:
# # To evaluate with the desired values for q, dq and ddq:
# q_d = [pi/2, pi/4, 3]
# dq_d = [30, 20, 13]
# ddq_d = [5, 3, 1]

# # Prestar atenção, a ordem de substituição é: ddq, dq, q
# #   Se apenas q for setada, dq e ddq são calculados automaticamente, pois são suas derivadas. (i.e., se colocar constantes, dq e ddq são ZERO)
# #   Se apenas dq for setada, ddq é calculada automaticamente, pois é sua derivada. (q não sofre alteração)
# #   Se todos os valores forem setados, segue-se a ordem de substituição, para que o sistema não identifique automaticamente os valores das derivadas e considere os valores dados.
# #   Tomar cuidado quando for substituir.
# # Interessante: podemos colocar uma trajetória desejada em função do tempo para seguir q_d(t), que os cálculos consideram as derivadas automaticamente: ver Exemplo 2
# # Exemplo 1
# r.eval_dinamics(tau=tau, q_d=q_d, dq_d=dq_d, ddq_d=ddq_d)

# # Exemplo 2: 
# # Trajetória plano X-Y [r.t, exp(-r.t), 0] (para substituir valores de t, basta: <res>.subs('t', valor) ou <res>.subs(r.t, valor))
# r.eval_dinamics(tau=tau, q_d=[r.t, exp(-r.t), 0]).subs('t', 4)   # .subs('t', 4)